## AI Hub 데이터를 활용한 KoGPT2 학습

<font color="red">이전 실습 파일에서 aihub_text.csv 파일 데이터 전처리를 하고 해당 파일로 실습을 진행합니다.

실습 중 다음의 오류가 발생한다면 colab 런타임을 재실행 해주세요.
NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
</font>

<img src="https://aihub.or.kr/web-nas/aihub21/files/editor/2022/06/13dabc7c0e4042e5a7d6c23c3f7422ef.png">

* https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=582
* https://huggingface.co/skt/kogpt2-base-v2
* https://github.com/SKT-AI/KoGPT2

In [ ]:
# HuggingFace transformer 패키지 설치
# !pip install -U -qq transformers

In [ ]:
# GPU 사용 여부 확인
# 'cuda:0', 'cpu'


## 사전 학습 모델 로드
* model_ckpt = "skt/kogpt2-base-v2":
    * model_ckpt 변수에는 KoGPT2의 체크포인트 경로를 지정합니다. "skt/kogpt2-base-v2"는 사전학습된 KoGPT2 모델 중 기본(베이스) 버전을 의미합니다.
* tokenizer = PreTrainedTokenizerFast.from_pretrained(model_ckpt, ...)
    * PreTrainedTokenizerFast.from_pretrained() 함수를 사용하여 미리 학습된 KoGPT2의 토크나이저를 불러옵니다.
    * bos_token, eos_token, unk_token, pad_token, mask_token을 지정하여 특수 토큰들을 설정합니다. `<s>, </s>, <unk>, <pad>, <mask>`가 순서대로 각각 지정되었습니다.
* model = GPT2LMHeadModel.from_pretrained(model_ckpt)
    * GPT2LMHeadModel.from_pretrained() 함수를 사용하여 미리 학습된 KoGPT2의 언어 모델을 불러옵니다.

In [ ]:
# KoGPT2 tokenizer, model 불러오기


## 텍스트 파일 로드

In [ ]:
# pandas


In [ ]:
# 상위 아티클과 요약문 보기
# raw_data = df


## 데이터셋 로드

* SummaryDataset 클래스:
    * tokenizer: 텍스트 데이터를 토큰화하기 위해 사용되는 토크나이저입니다. 이 토크나이저는 Hugging Face의 Transformers 라이브러리에서 불러온 KoGPT2의 토크나이저입니다.
    * raw_data: 원시 데이터셋 입니다.
    * max_len: 각 데이터의 최대 길이를 정의하는 변수입니다. 생성된 토큰 시퀀스가 이 길이를 초과하면 최대 길이로 자르게 됩니다.
    * data: 토큰화된 데이터를 저장하는 리스트입니다.

* `__init__` 메서드:
    * 클래스 객체가 생성될 때 호출되는 초기화 메서드입니다.
    * max_len, tokenizer, SEP_CHARS, data 등의 멤버 변수를 초기화하고 _load_and_build 메서드를 호출하여 데이터셋을 구성합니다.

* `_load_and_build` 메서드:
    * 데이터셋을 로드하고 토큰화하여 전처리하는 메서드입니다.
    * 텍스트가 최대 길이인 max_len보다 길다면, 최대 길이에 맞게 자르고 토큰 시퀀스를 구성합니다.
    * 텍스트가 최대 길이보다 짧다면, 부족한 부분을 패딩 토큰으로 채워서 토큰 시퀀스를 구성합니다.

* `__len__` 메서드:
    * 데이터셋의 총 샘플 개수를 반환하는 메서드입니다. len(dataset) 형태로 호출되며, len(dataset.data)와 같은 결과를 반환합니다.

* `__getitem__` 메서드:
    * 데이터셋의 인덱스에 해당하는 샘플을 반환하는 메서드입니다. dataset[idx] 형태로 호출되며, dataset.data[idx]와 같은 결과를 반환합니다. 이때, 데이터는 PyTorch의 텐서로 반환됩니다.

In [ ]:
from torch.utils.data import Dataset, DataLoader

class SummaryDataset(Dataset):
    def __init__(self, tokenizer, raw_data, max_len=500):
        # tokenizer, max_len, SEP_CHARS, data


    def _process_row(self, row):
        """
        기사와 요약을 토크나이징하고, 토큰 길이가 max_len을 초과하지 않도록 조정합니다.
        토크나이징된 기사와 요약 사이에 구분자(SEP_CHARS)를 끼워 넣습니다.
        필요한 경우 패딩을 추가하여 모든 데이터가 동일한 길이를 가지도록 합니다.
        """
        # tokenized_article
        # tokenized_summary
        # tokenized_sep_chars
        # total_length


        if total_length > self.max_len:
            tokenized_article = tokenized_article[:self.max_len - total_length]

        padding_length = self.max_len - len(tokenized_article) - len(tokenized_sep_chars) - len(tokenized_summary) - 1
        return torch.tensor(tokenized_article
                            + tokenized_sep_chars
                            + tokenized_summary
                            + [self.tokenizer.eos_token_id]
                            + ([self.tokenizer.pad_token_id] * padding_length)
                            )

    def _load_and_build(self, raw_data):
        self.data = raw_data.apply(self._process_row, axis=1).tolist()
        return self.data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
# dataset = SummaryDataset()
# dataloader = DataLoader()

In [ ]:
# 첫 번째 배치 가져오기
# first_batch

# 가져온 배치 출력


In [ ]:
# 첫 번째 배치 heatmap 시각화
import seaborn as sns


## 학습

In [ ]:
#  모델의 어떤 부분이 가장 많은 메모리를 사용하는지 식별
# print(torch.cuda.memory_summary())

In [ ]:
# GPU 캐시를 비워 모델에서 사용할 수 있는 메모리를 확보할 수 있습니다.
# torch.cuda.empty_cache()

In [ ]:
EPOCHS = 3

# model
# optimizer

# epoch > dataloader > torch.set_grad_enabled()
            # optimizer
            # batch
            # output
            # loss
            # backward
            # optimizer.step()

            # if idx % 100 == 0:
            #     print("epoch: {}, step: {:3d}, loss: {:.3f}".format(epoch+1, idx, loss))

## 생성에 사용할 입력 문서 확인

In [ ]:
# 샘플 문서
sample_idx = 0
sample_text = df.loc[sample_idx, "article"]

print(sample_text)
# 샘플 문서 + " TL;DR "
sample_text = sample_text + " TL;DR "

# 입력 값을 tokenize
# tokenized_sample

## 생성
* repetition_penalty
    * 정의: repetition_penalty는 모델이 이전에 생성한 토큰을 반복하는 것을 억제하는 데 사용됩니다.
    * 작동 방식: 이 매개변수는 이미 생성된 토큰의 확률을 인위적으로 감소시켜, 모델이 같은 단어나 구를 반복적으로 생성하는 것을 방지합니다. 값이 1보다 크면 반복되는 토큰의 확률이 감소하고, 1이면 영향을 주지 않습니다.
* no_repeat_ngram_size
    * 정의: no_repeat_ngram_size는 생성된 텍스트에서 n-gram의 반복을 방지합니다.
    * 작동 방식: 이 매개변수는 지정된 크기(n)의 n-gram이 생성된 텍스트 내에서 반복되는 것을 방지합니다. 예를 들어, no_repeat_ngram_size=2로 설정하면 동일한 2-gram이 생성된 텍스트 내에서 반복되지 않도록 합니다.
* top-k 샘플링
    * 정의: top-k 샘플링은 모델이 다음 토큰을 예측할 때 가능성이 높은 상위 k개의 토큰 중에서 무작위로 선택하는 방법입니다.
    * 작동 방식: 모델이 다음 토큰에 대한 확률 분포를 계산한 후, 가장 높은 확률을 가진 상위 k개의 토큰을 선택합니다. 그런 다음 이 k개의 토큰 중 하나를 무작위로 선택하여 텍스트를 생성합니다.
    * 목적: top-k 샘플링은 생성된 텍스트의 다양성을 보장하면서도, 너무 낮은 확률을 가진 토큰을 배제하여 품질을 유지합니다.
* Top-p (Nucleus) 샘플링
    * 정의: top-p 샘플링은 확률 분포의 누적 확률이 p 이상이 되는 최소한의 토큰 집합을 선택하는 방법입니다.
    * 작동 방식: 먼저 모델이 다음 토큰에 대한 확률 분포를 계산합니다. 그런 다음 높은 확률의 토큰부터 순차적으로 추가하면서, 누적 확률이 p 이상이 되는 순간의 토큰 집합을 선택합니다. 이 집합 내에서 토큰이 무작위로 선택됩니다.
    * 목적: top-p 샘플링은 다양성과 품질 사이의 균형을 맞추는 데 효과적입니다. top-k와 달리, p 값에 따라 선택되는 토큰의 수가 동적으로 변경됩니다.

* temperature
    * 정의: temperature는 생성 과정에서 확률 분포의 '온도'를 조절하여 결과의 무작위성과 예측 가능성을 조절합니다.
    * 작동 방식: 낮은 temperature 값(예: 0.7)은 더 예측 가능하고 일관된 텍스트를 생성하도록 합니다. 반면 높은 값(예: 1 이상)은 더 다양하고 예측하기 어려운 텍스트를 생성합니다. temperature는 확률 분포를 '평탄화'하거나 '날카롭게' 만듦으로써 선택의 다양성을 조절합니다.


In [ ]:
# 학습한 모델에 입력
model.eval()

output = model.generate(
    tokenized_sample,
    max_length=2000,               # 생성할 텍스트의 최대 길이
    repetition_penalty=5.0,        # 반복 패널티
    no_repeat_ngram_size=2,        # 반복되는 n-gram 크기 제한
    top_k=50,                      # top-k 샘플링
    top_p=0.95,                    # top-p 샘플링 (nucleus sampling)
    temperature=0.7,               # 생성 다양성 조절
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    use_cache=True
)

# 출력 값에 포함된 입력 값 부분을 제외하고 출력
# output_decode

In [ ]:
# df.loc[sample_idx, "summary"]

## ROUGE-N (N-gram Co-occurrence Statistics)

* ROUGE-N은 생성된 요약과 참조 요약 간의 N-gram 일치를 측정합니다.
    * 가장 일반적인 것은 ROUGE-1(단일 단어 일치)과 ROUGE-2(2-gram 일치)입니다.
    * ROUGE-N 스코어는 정밀도(Precision), 재현율(Recall), F1 스코어의 세 가지 값을 제공합니다.
* ROUGE-L (Longest Common Subsequence):
    * ROUGE-L은 생성된 요약과 참조 요약 간의 최장 공통 부분 수열(Longest Common Subsequence)을 측정합니다.
    * 이 메트릭은 단어의 순서를 고려하여 요약의 질을 평가합니다.
* ROUGE-S:
    * ROUGE-S(또는 ROUGE-SU)는 단어 순서가 덜 중요한 경우에 사용되며,2-gram이지만 단어 간에 다른 단어들이 삽입될 수 있습니다.


* ROUGE(Rouge-Oriented Understudy for Gisting Evaluation) 스코어를 직접 계산하기 위해서는 생성된 요약과 참조 요약 간의 n-gram 일치를 계산해야 합니다.

* ROUGE-N 스코어 계산 방법
ROUGE-N 스코어는 참조 요약과 생성된 요약 간의 n-gram 오버랩을 측정합니다. ROUGE-1은 단일 단어(1-gram)에 대한 오버랩을, ROUGE-2는 두 단어 쌍(2-gram)에 대한 오버랩을 측정합니다.

In [ ]:
def ngram(token_list, n):
    return set(zip(*[token_list[i:] for i in range(n)]))

def rouge_n_score(candidate, reference, n=1):
    candidate_ngrams = ngram(candidate.split(), n)
    reference_ngrams = ngram(reference.split(), n)
    overlap_ngrams = candidate_ngrams.intersection(reference_ngrams)
    return len(overlap_ngrams) / len(candidate_ngrams), len(overlap_ngrams) / len(reference_ngrams)

def calculate_f1_score(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)


def rouge_score(candidate, reference):
    # ROUGE-1 스코어 계산
    precision_1, recall_1 = rouge_n_score(candidate, reference, n=1)
    f1_score_1 = calculate_f1_score(precision_1, recall_1)
    # ROUGE-2 스코어 계산
    precision_2, recall_2 = rouge_n_score(candidate, reference, n=2)
    f1_score_2 = calculate_f1_score(precision_2, recall_2)

    print(f"ROUGE-1: Precision: {precision_1}, Recall: {recall_1}, F1: {f1_score_1}")
    print(f"ROUGE-2: Precision: {precision_2}, Recall: {recall_2}, F1: {f1_score_2}")


# 예시 텍스트
candidate = "산책을 하기 좋은 날씨는 맑고 화창한 날입니다."
reference = "산책을 하기 좋은 날씨는 맑고 화창한 날입니다."
# reference = "맑고 화창한 날은 산책하기 좋은 날씨입니다."

print(ngram(candidate, 1))
print(ngram(reference, 1))
rouge_score(candidate, reference)

## 생성한 텍스트의 ROUGE 점수 측정

## 다른 거대 모델에 비해 낮은 성능을 갖는 이유
* 모델 크기와 복잡성: 일반적으로, 더 큰 모델(더 많은 매개변수를 가진 모델)은 더 작은 모델보다 더 많은 양의 데이터를 학습하고, 더 복잡한 패턴을 인식할 수 있습니다. 따라서, 거대한 LLMs는 종종 더 작은 모델보다 더 우수한 성능을 보입니다.

* 학습 데이터: 모델의 성능은 사용된 학습 데이터의 양과 질에 크게 의존합니다. 더 다양하고 방대한 데이터셋으로 학습된 모델은 일반적으로 더 좋은 성능을 나타냅니다.

* 사전 학습 및 미세 조정: 많은 대형 언어 모델들은 먼저 대규모 데이터셋에서 사전 학습된 후, 특정 작업에 대해 미세 조정됩니다. 사전 학습과 미세 조정 과정의 효율성이 모델 성능에 큰 영향을 미칩니다.

* 아키텍처의 차이: 모델의 아키텍처 또한 성능에 영향을 줍니다. 변형된 Transformer 구조, 새로운 어텐션 메커니즘, 레이어 정규화 방법 등은 성능에 큰 차이를 만들 수 있습니다.

* 최적화 및 하이퍼파라미터 조정: 학습 과정에서 사용되는 최적화 기법과 하이퍼파라미터(학습률, 배치 크기 등)의 설정은 모델의 최종 성능에 중요한 역할을 합니다.

* 작업 특이성: 특정 모델이 특정 작업에 대해 더 높은 성능을 보이는 경우가 있습니다. 예를 들어, 어떤 모델은 자연어 이해(NLU)에서 뛰어난 반면, 다른 모델은 자연어 생성(NLG)에서 더 좋은 성능을 보일 수 있습니다.

* 업데이트와 유지 관리: 지속적인 업데이트와 유지 관리는 모델 성능을 지속적으로 개선하는 데 중요합니다. 최신 연구 결과나 기술의 적용이 성능 향상에 기여할 수 있습니다.